# Research Question Tests
How Is the misalia and tawasi intervalic structure on each san’a from a nawba?
This one would be extract each misalia/tawasi as segment and extract intervalic structure(intervals) and then check for how equal they are.

By Rafa: How is the misalia and tawasi Characterization on the nawbas according to Amin?

In [1]:
from music21 import *
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import pprint

jsonFile = '../arab-andalusian-scores-lines.json'

In [2]:
with open (jsonFile, 'r', encoding='utf-8') as f:
    annotations = json.load(f)

In [3]:
sections = {}
sectionNames = ['mišālia','tawāšī']

for score in annotations:
    mbid = score['mbid']
    sections[mbid] = {}
    
    for section in score['sections']:
        name = section['name']
        if not name in sectionNames:
            continue
        if name not in sections[mbid]:
            sections[mbid][name] = []
            
        sections[mbid][name].append((section['start'], section['end']))
        
pprint.pprint(sections)

{'7448d9c2-5261-4e70-bd98-6ed8416f908f': {'mišālia': [(2.5, 256.0)],
                                          'tawāšī': [(260.0, 536.0)]},
 '8842c1f0-e261-4069-bd59-768bb9a3315c': {'mišālia': [(0.0, 208.0)],
                                          'tawāšī': [(213.0, 259.5)]},
 'a451a7fc-c53f-462a-b3fc-4377bb588105': {'mišālia': [(0.0, 208.0)]},
 'b11237b9-d45b-4b3a-a97b-ab7d198f927f': {'mišālia': [(0.0, 208.0)],
                                          'tawāšī': [(213.0, 290.5)]},
 'f7bcb9af-6abb-4192-ae3d-37fa811034ce': {'mišālia': [(0.0, 208.0)],
                                          'tawāšī': [(213.0, 290.5)]}}


In [4]:
#intervals = {name:{} for name in sectionNames}
final_intervals = {}
histogram = {}
centos = {}
centos_template = {
    'C2': ['E', 'F', 'G'], 
    'C3': ['B', 'D', 'C'], 
    'C4': ['B', 'C'], 
    'C5': ['E','F','G','E','F','E','D','C'],
    'C6': ['G', 'F', 'E'], 
}

def extract_interval_from_note(n):
    return interval.Interval(n, n.next())

def extract_intervals(section_notes):
    intervals = {}
    for n in section_notes[:-1]:
        if not n.isNote or not n.next().isNote:
            continue
        
        itv = extract_interval_from_note(n)
        intervals[itv.name] = intervals.get(itv.name, 0) + 1

    return intervals

def is_centos_2(note):
    temp = [note.name, note.next().name, note.next().next().name]
    return centos_template['C2'] == temp
def is_centos_6(note):
    temp = [note.name, note.next().name, note.next().next().name]
    return centos_template['C6'] == temp
def is_centos_3(note):
    temp = [note.name, note.next().name, note.next().next().name]
    return centos_template['C3'] == temp
def is_centos_4(note):
    temp = [note.name, note.next().name]
    return centos_template['C4'] == temp
    
def extract_intervals_from_sections_analise(sections):
    
    for mbid in sections.keys():
        print('Loading %s.xml' %(mbid))
        s = converter.parse('../%s.xml'%(mbid))
        #get allnotes and rests
        nr = s.flat.notesAndRests.stream()
        score = sections[mbid]
        for sectionName, segments in score.items():
            print('Analizing - %s' %sectionName)
            current_section = score[sectionName]
            for score_section in current_section:
                start = score_section[0]
                end = score_section[1]
                section_notes = nr.getElementsByOffset(start, end).stream()
                '''for n in section_notes[:-1]:
                    if not n.isNote or not n.next().isNote:
                        continue
                    ## Create key if it does not exist
                    if not mbid in intervals:
                        intervals.update({mbid:{}})
                        
                    if not sectionName in intervals[mbid]:
                        intervals[mbid].update({sectionName:{}})
                        
                    itv = extract_interval_from_note(n)
                    intervals[mbid][sectionName][itv.name] = intervals[mbid][sectionName].get(itv.name, 0) + 1
                '''
                
                ## INTERVALS RELATED
                #if not mbid in final_intervals:
                #    final_intervals.update({mbid:{}})
                        
                #if not sectionName in final_intervals[mbid]:
                #    final_intervals[mbid].update({sectionName:{}})
                        
               # final_intervals[mbid][sectionName].update(extract_intervals(section_notes))
                
                ## HISTOGRAM OF NOTE DURATION (ask for wich note duration is the best)
                for n in section_notes[:-1]:
                    if not n.isNote or not n.next().isNote:
                        continue
                    if not mbid in histogram:
                        histogram.update({mbid:{}})
                        
                    if not sectionName in histogram[mbid]:
                        histogram[mbid].update({sectionName:{}})
                        
                    note_name = n.name
                    note_duration = n.duration.quarterLength
                    #print(note_duration)
                    histogram[mbid][sectionName].update({note_name: histogram[mbid][sectionName].get(note_name, 0) + note_duration})
                
               
                
                ## CENTOS
                for n in section_notes[:-1]:
                    if not n.isNote:
                        continue
                        
                    if not mbid in centos:
                        centos.update({mbid:{}})
                        
                    if not sectionName in centos[mbid]:
                        centos[mbid].update({sectionName:{}})
                    #print('note:%s , next: %s, nextnext:%s'%(n.name, n.next().name, n.next().next().name))
                    #print(is_centos_1(n))
                    if is_centos_1(n):
                        centos[mbid][sectionName].update({'C1': centos[mbid][sectionName].get('C1', 0) + 1})
                    
                    if is_centos_2(n):
                        centos[mbid][sectionName].update({'C2': centos[mbid][sectionName].get('C2', 0) + 1})
                        
                    if is_centos_3(n):
                        #print('note:%s , next: %s, nextnext:%s'%(n.name, n.next().name, n.next().next().name))
                        centos[mbid][sectionName].update({'C3': centos[mbid][sectionName].get('C3', 0) + 1})
                        
                    if is_centos_4(n):
                        centos[mbid][sectionName].update({'C4': centos[mbid][sectionName].get('C4', 0) + 1})
                        
                        
                    if is_centos_6(n):
                        #print('note:%s , next: %s, nextnext:%s'%(n.name, n.next().name, n.next().next().name))
                        centos[mbid][sectionName].update({'C6': centos[mbid][sectionName].get('C6', 0) + 1})
                        
                    
                
                
                        
extract_intervals_from_sections_analise(sections)
pprint.pprint(histogram)
pprint.pprint(final_intervals)
pprint.pprint(centos)

Loading 7448d9c2-5261-4e70-bd98-6ed8416f908f.xml
Analizing - mišālia
Analizing - tawāšī
Loading 8842c1f0-e261-4069-bd59-768bb9a3315c.xml
Analizing - mišālia
Analizing - tawāšī
Loading a451a7fc-c53f-462a-b3fc-4377bb588105.xml
Analizing - mišālia
Loading b11237b9-d45b-4b3a-a97b-ab7d198f927f.xml
Analizing - mišālia
Analizing - tawāšī
Loading f7bcb9af-6abb-4192-ae3d-37fa811034ce.xml
Analizing - mišālia
Analizing - tawāšī
{'7448d9c2-5261-4e70-bd98-6ed8416f908f': {'mišālia': {'A': 23.0,
                                                      'B': 16.5,
                                                      'B-': 4.0,
                                                      'C': 13.5,
                                                      'D': 5.0,
                                                      'E': 19.5,
                                                      'F': 15.0,
                                                      'G': 18.5},
                                          'tawāšī': {'A': 7